In [4]:
import os
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install keras-facenet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10367 sha256=1fad45be948701f2c47e7b19767559b50def3f61679930e599d9d94052add2ea
  Stored in directory: /root/.cache/pip/wheels/99/94/dd/cb1a65a7440ba6d508bd24346c15af0b1d24ff8b1cdb1c9959
Successfully built keras-facenet


In [2]:
!pwd

/content


In [3]:
!ls

sample_data


In [12]:
import os
import cv2
import csv
import pickle
import numpy as np
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Load pretrained FaceNet model (MTCNN + embeddings)
embedder = FaceNet()
print("✅ FaceNet model loaded")

# Load stored FaceNet embeddings and label encoder
data = np.load("/content/embeddings_dataset.npz")
stored_embeddings = data['arr_0']  # normalized embeddings
stored_labels = data['arr_1']      # integer-encoded labels

with open("/content/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

test_dir = "/content/drive/MyDrive/testing_dataset"
output_csv = "/content/facenet_test_results_2.csv"
vis_output_dir = "/content/facenet_prediction_visuals"
os.makedirs(vis_output_dir, exist_ok=True)

# Open CSV file for one-entry-per-image results
with open(output_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([
        'Image',
        'Num_Faces',
        'Predictions',
        'Similarities',
        'Match_Statuses',
        'Bounding_Boxes'
    ])

    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"[WARN] Skipping unreadable image: {img_name}")
            continue

        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = embedder.extract(rgb_img, threshold=0.95)

        if not results:
            writer.writerow([img_name, 0, "No face detected", "", "", ""])
            continue

        predictions = []
        similarities = []
        statuses = []
        boxes = []

        for face in results:
            emb = face['embedding'].reshape(1, -1)
            sims = cosine_similarity(emb, stored_embeddings)[0]
            best_idx = np.argmax(sims)
            similarity = sims[best_idx]
            predicted_name = label_encoder.inverse_transform([stored_labels[best_idx]])[0]

            is_match = similarity > 0.75
            label = f"{predicted_name} ({similarity:.2f})" if is_match else "Unknown"
            status = "Matched" if is_match else "Unknown"

            x, y, w, h = face['box']
            x, y = abs(x), abs(y)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            predictions.append(label)
            similarities.append(f"{similarity:.2f}")
            statuses.append(status)
            boxes.append(f"({x}, {y}, {w}, {h})")

        writer.writerow([
            img_name,
            len(results),
            "; ".join(predictions),
            "; ".join(similarities),
            "; ".join(statuses),
            "; ".join(boxes)
        ])

        output_path = os.path.join(vis_output_dir, f"annotated_{img_name}")
        cv2.imwrite(output_path, img)

print(f"Inference complete. One-row-per-image results saved to: {output_csv}")

✅ FaceNet model loaded


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
✅ Inference complete. One-row-per-image results saved to: /content/facenet_test_results_2.csv


In [ ]:
import shutil

shutil.make_archive('/content/facenet_prediction_visuals_2', 'zip', '/content/facenet_prediction_visuals_2')

from google.colab import files

files.download('/content/facenet_prediction_visuals_2.zip')